In [2]:
import numpy as np
import pandas as pd
import os
from functools import reduce

In [292]:
df1 = pd.read_csv('../Data/Clean/USAFacts/COVIDcounts_county_clean.csv', index_col = 0)
print(df1.shape)
df1.head()

(3142, 895)


,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,5813,5818,5825,5832,5832,5832,5857,5863,5880,5891
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,6630,6648,6654,6663,6663,6663,6679,6687,6700,6704
5,1009,Blount County,AL,1,0,0,0,0,0,0,...,15246,15249,15272,15283,15283,15283,15324,15335,15351,15370


In [293]:
for index, column in enumerate(df1.columns[:]):
    if column.startswith('2020-12-30'):
        labs20 = column
    if column.startswith('2021-12-30'):
        labs21 = column
    if column.startswith('2022-06-30'):
        labs22 = column

print(labs20)
print(labs21)
print(labs22)

2020-12-30
2021-12-30
2022-06-30


In [294]:
df1 = pd.DataFrame(np.repeat(df1.values, 3, axis = 0), columns = df1.columns)
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16313,16330,16360,16400,16400,16400,16468,16496,16520,16536
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58009,58169,58293,58372,58372,58372,58595,58695,58796,58905


In [295]:
df1['year'] = np.nan
#df1['year'] = df1.year.astype('int64').fillna()
df1['year'] = df['year'].astype('int64').fillna(np.nan)
for index in df1.index:
    if index % 3 == 0:
        df1.iloc[index, -1] = 2020
    elif index % 3 == 1:
        df1.iloc[index, -1] = 2021
    else:
        df1.iloc[index, -1] = 2022
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2020
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2021
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16330,16360,16400,16400,16400,16468,16496,16520,16536,2022
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58169,58293,58372,58372,58372,58595,58695,58796,58905,2020
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58169,58293,58372,58372,58372,58595,58695,58796,58905,2021


In [296]:
df1['covid_cases'] = np.nan
#df1['covid_cases'] = df1['covid_cases'].astype('int64').fillna(np.nan)
df1.head()

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year,covid_cases
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2020,NaN
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2021,NaN
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2022,NaN
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2020,NaN
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2021,NaN


In [297]:
for index in df1.index:
    if index % 3 == 0:
        df1.iloc[index,-1] = df1.loc[index,labs20]
    elif index % 3 == 1:
        df1.iloc[index,-1] = df1.loc[index,labs21]
    else:
        df1.iloc[index,-1] = df1.loc[index,labs22]
df1

,fips,county_name,state,statefips,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28,2022-06-29,2022-06-30,year,covid_cases
0,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2020,4164.0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2021,10953.0
2,1001,Autauga County,AL,1,0,0,0,0,0,0,...,16360,16400,16400,16400,16468,16496,16520,16536,2022,16536.0
3,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2020,13392.0
4,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,58293,58372,58372,58372,58595,58695,58796,58905,2021,39583.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9421,56043,Washakie County,WY,56,0,0,0,0,0,0,...,2490,2490,2490,2490,2490,2482,2482,2482,2021,1880.0
9422,56043,Washakie County,WY,56,0,0,0,0,0,0,...,2490,2490,2490,2490,2490,2482,2482,2482,2022,2482.0
9423,56045,Weston County,WY,56,0,0,0,0,0,0,...,1641,1641,1641,1641,1641,1643,1643,1643,2020,475.0
9424,56045,Weston County,WY,56,0,0,0,0,0,0,...,1641,1641,1641,1641,1641,1643,1643,1643,2021,1254.0


In [298]:
df = df1[['year', 'fips','covid_cases', 'county_name']]
print(df.dtypes)
df[['year','fips','covid_cases']] = df[['year','fips','covid_cases']].apply(pd.to_numeric)
print(df.dtypes)
df

year             int64
fips            object
covid_cases    float64
county_name     object
dtype: object
year             int64
fips             int64
covid_cases    float64
county_name     object
dtype: object


/var/folders/jb/2grr3hjx7fng_kz01_rqpskm0000gn/T/ipykernel_12863/161515683.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['year','fips','covid_cases']] = df[['year','fips','covid_cases']].apply(pd.to_numeric)


,year,fips,covid_cases,county_name
0,2020,1001,4164.0,Autauga County
1,2021,1001,10953.0,Autauga County
2,2022,1001,16536.0,Autauga County
3,2020,1003,13392.0,Baldwin County
4,2021,1003,39583.0,Baldwin County
...,...,...,...,...
9421,2021,56043,1880.0,Washakie County
9422,2022,56043,2482.0,Washakie County
9423,2020,56045,475.0,Weston County
9424,2021,56045,1254.0,Weston County


In [299]:
df.to_csv('../Data/Intermediate/COVIDcounty_cases_processed.csv')

In [300]:
dp03, dp05 = [], []
path = '../Data/Clean/ACS'
for file in os.listdir(path):
    if file.find('DP03') != -1:
        print(file)
        dp03.append(pd.read_csv(os.path.join(path, file), index_col = 0))
    if file.find('DP05') != -1:
        print(file)
        dp05.append(pd.read_csv(os.path.join(path, file), index_col = 0))

i = 0
while i != len(dp03):
    dp03[i].drop(columns = ['geographic_area_name','id'], inplace = True)
    print('03: ', dp03[i].shape)
    dp05[i].drop(columns = ['Geographic Area Name','id'], inplace = True)
    print('05: ',dp05[i].shape)
    i += 1

ACSDP052018_clean.csv
ACSDP032016_clean.csv
ACSDP052015_clean.csv
ACSDP052020_clean.csv
ACSDP032010_clean.csv
ACSDP052013_clean.csv
ACSDP052019_clean.csv
ACSDP032017_clean.csv
ACSDP052014_clean.csv
ACSDP032011_clean.csv
ACSDP052012_clean.csv
ACSDP032014_clean.csv
ACSDP032019_clean.csv
ACSDP052017_clean.csv
ACSDP032012_clean.csv
ACSDP052011_clean.csv
ACSDP032015_clean.csv
ACSDP032020_clean.csv
ACSDP032018_clean.csv
ACSDP052016_clean.csv
ACSDP032013_clean.csv
ACSDP052010_clean.csv
03:  (3220, 20)
05:  (3220, 34)
03:  (3221, 20)
05:  (3220, 33)
03:  (3220, 20)
05:  (3221, 34)
03:  (3221, 20)
05:  (3221, 33)
03:  (3220, 20)
05:  (3220, 34)
03:  (3220, 20)
05:  (3220, 33)
03:  (3221, 20)
05:  (3221, 19)
03:  (3220, 20)
05:  (3220, 34)
03:  (3221, 20)
05:  (3221, 19)
03:  (3220, 20)
05:  (3220, 33)
03:  (3221, 20)
05:  (3221, 19)


In [301]:
i = 2
tot_dp03 = pd.concat([dp03[0], dp03[1]], ignore_index = True)
tot_dp05 = pd.concat([dp05[0], dp05[1]], ignore_index = True)

while i != len(dp03):
    tot_dp03 = pd.concat([tot_dp03, dp03[i]], ignore_index = True)
    tot_dp05 = pd.concat([tot_dp05, dp05[i]], ignore_index = True)
    i += 1

print(tot_dp03.shape)
print(tot_dp05.shape)
tot_dp03

(35425, 31)
(35425, 42)


,per_capita_income_2016,employment_pop_ratio,unemployment_rate,pct_ind_agriculture_forestry_fishing_and_hunting_and_mining,pct_ind_construction,pct_ind_manufacturing,pct_ind_wholesale_trade,pct_ind_retail_trade,pct_ind_transportation_and_warehousing_and_utilities,pct_ind_information,...,per_capita_income_2017,per_capita_income_2011,per_capita_income_2014,pct_employment_status!!population_16_years_and_over!!in_labor_force!!civilian_labor_force!!unemployed,per_capita_income_2019,per_capita_income_2012,per_capita_income_2015,per_capita_income_2020,per_capita_income_2018,per_capita_income_2013
0,26168.0,56.8,5.6,1.2,4.8,14.0,3.1,10.4,6.6,1.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,28069.0,54.7,6.3,1.4,7.7,9.2,3.0,15.0,4.9,1.4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17249.0,41.9,12.8,4.5,6.3,22.4,2.8,11.6,6.4,0.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,18988.0,45.2,7.1,3.9,8.2,21.1,1.5,10.2,6.7,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,21033.0,46.9,6.0,2.9,9.5,15.6,3.7,11.2,7.5,1.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35420,NaN,30.2,NaN,0.5,4.8,15.2,2.6,14.7,3.6,1.2,...,NaN,NaN,NaN,4.7,NaN,NaN,NaN,NaN,NaN,9017.0
35421,NaN,41.6,NaN,5.5,12.6,7.5,1.2,5.4,5.4,0.4,...,NaN,NaN,NaN,4.7,NaN,NaN,NaN,NaN,NaN,8370.0
35422,NaN,35.6,NaN,5.9,7.3,17.4,0.7,9.0,2.2,1.1,...,NaN,NaN,NaN,14.0,NaN,NaN,NaN,NaN,NaN,7997.0
35423,NaN,29.2,NaN,1.7,6.2,17.5,0.8,13.9,3.0,0.3,...,NaN,NaN,NaN,8.4,NaN,NaN,NaN,NaN,NaN,7956.0


In [302]:
tot_dp05.columns

Index(['pct_one_or_more_race', 'pct_one_or_more_race_white',
       'pct_one_or_more_race_black', 'pct_one_or_more_race_native_american',
       'pct_one_or_more_race_asian',
       'pct_one_or_more_race_native_hawaiian_pacific_islander',
       'pct_one_or_more_race_other_race', 'pct_ethnicity_hispanic',
       'pct_sex_age_under_5', 'pct_sex_age_5_to_9', 'pct_sex_age_10_to_14',
       'pct_sex_age_15_to_19', 'pct_sex_age_20_to_24', 'pct_sex_age_25_to_34',
       'pct_sex_age_35_to_44', 'pct_sex_age_45_to_54', 'pct_sex_age_55_to_59',
       'pct_sex_age_60_to_64', 'pct_sex_age_65_to_74', 'pct_sex_age_75_to_84',
       'pct_sex_age_85+', 'median_age', 'pct_sex_age_16+', 'pct_sex_age_18+',
       'pct_sex_age_21+', 'pct_sex_age_62+', 'pct_sex_age_65+',
       'pct_sex_age_18+_male', 'pct_sex_age_18+_female',
       'pct_sex_age_65+_male', 'pct_sex_age_65+_female', 'year', 'total_pop',
       'fips', 'pct_sex_age_male', 'pct_sex_age_female', 'pct_white',
       'pct_black', 'pct_native_a

In [303]:
tot_dp05

,pct_one_or_more_race,pct_one_or_more_race_white,pct_one_or_more_race_black,pct_one_or_more_race_native_american,pct_one_or_more_race_asian,pct_one_or_more_race_native_hawaiian_pacific_islander,pct_one_or_more_race_other_race,pct_ethnicity_hispanic,pct_sex_age_under_5,pct_sex_age_5_to_9,...,total_pop,fips,pct_sex_age_male,pct_sex_age_female,pct_white,pct_black,pct_native_american,pct_asian,pct_native_hawaiian_pacific_islander,pct_other_race
0,55200,78.6,19.9,0.8,1.6,0.2,0.8,2.8,5.9,7.3,...,55200,1001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,208107,87.8,9.8,1.5,1.4,0.0,1.3,4.5,5.6,5.6,...,208107,1003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,25782,48.3,48.5,1.2,0.6,0.0,3.1,4.3,5.4,5.6,...,25782,1005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22527,77.4,22.7,0.4,0.2,0.0,0.0,2.4,5.7,5.2,...,22527,1007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,57645,97.1,1.9,1.2,0.5,0.1,0.8,9.1,6.0,6.3,...,57645,1009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35420,60436,NaN,NaN,NaN,NaN,NaN,NaN,98.6,NaN,NaN,...,60436,72145,48.2,51.8,94.6,3.9,0.3,0.1,0.0,2.4
35421,9313,NaN,NaN,NaN,NaN,NaN,NaN,95.7,NaN,NaN,...,9313,72147,49.6,50.4,71.6,22.3,0.0,0.2,0.0,13.4
35422,26565,NaN,NaN,NaN,NaN,NaN,NaN,99.7,NaN,NaN,...,26565,72149,49.5,50.5,73.4,3.2,0.4,0.0,0.0,25.3
35423,38387,NaN,NaN,NaN,NaN,NaN,NaN,99.3,NaN,NaN,...,38387,72151,48.8,51.2,85.3,36.9,0.3,0.2,0.0,7.2


In [304]:
tot_dp05['median_age'] = pd.to_numeric(tot_dp05['median_age'], errors = 'coerce')
tot_dp03['health_insurance_rate'] = pd.to_numeric(tot_dp03['health_insurance_rate'], errors = 'coerce')
print(tot_dp05.dtypes)
print(tot_dp03.dtypes)

pct_one_or_more_race                                       int64
pct_one_or_more_race_white                               float64
pct_one_or_more_race_black                               float64
pct_one_or_more_race_native_american                     float64
pct_one_or_more_race_asian                               float64
pct_one_or_more_race_native_hawaiian_pacific_islander    float64
pct_one_or_more_race_other_race                          float64
pct_ethnicity_hispanic                                   float64
pct_sex_age_under_5                                      float64
pct_sex_age_5_to_9                                       float64
pct_sex_age_10_to_14                                     float64
pct_sex_age_15_to_19                                     float64
pct_sex_age_20_to_24                                     float64
pct_sex_age_25_to_34                                     float64
pct_sex_age_35_to_44                                     float64
pct_sex_age_45_to_54     

In [305]:
tot_dp03.to_csv('../Data/Intermediate/ACSDP03_merge.csv')
tot_dp05.to_csv('../Data/Intermediate/ACSDP05_merge.csv')

In [306]:
print(tot_dp03.shape)
print(tot_dp03.dtypes)
print(tot_dp05.dtypes)

(35425, 31)
per_capita_income_2016                                                                                   float64
employment_pop_ratio                                                                                     float64
unemployment_rate                                                                                        float64
pct_ind_agriculture_forestry_fishing_and_hunting_and_mining                                              float64
pct_ind_construction                                                                                     float64
pct_ind_manufacturing                                                                                    float64
pct_ind_wholesale_trade                                                                                  float64
pct_ind_retail_trade                                                                                     float64
pct_ind_transportation_and_warehousing_and_utilities                                

In [307]:
hps = pd.read_csv('../Data/Clean/HPS/hps_clean.csv', index_col = 0)
print(hps.columns)

Index(['year', 'area', 'pct_delayed_healthcare'], dtype='object')


In [308]:
hps['state_fips'] = np.nan
state = [1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 
         21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 
         38, 39, 40, 41, 42, 44, 45, 46, 47, 48, 49, 50, 51, 53, 54, 55, 56]
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming']

dic = {state[i]: us_states[i] for i in range(len(state))}

print(len(state))
print(len(us_states))
print(dic)

hps['state_fips'] = dic
hps

51
51
{1: 'Alabama', 2: 'Alaska', 4: 'Arizona', 5: 'Arkansas', 6: 'California', 8: 'Colorado', 9: 'Connecticut', 10: 'Delaware', 11: 'District of Columbia', 12: 'Florida', 13: 'Georgia', 15: 'Hawaii', 16: 'Idaho', 17: 'Illinois', 18: 'Indiana', 19: 'Iowa', 20: 'Kansas', 21: 'Kentucky', 22: 'Louisiana', 23: 'Maine', 24: 'Maryland', 25: 'Massachusetts', 26: 'Michigan', 27: 'Minnesota', 28: 'Mississippi', 29: 'Missouri', 30: 'Montana', 31: 'Nebraska', 32: 'Nevada', 33: 'New Hampshire', 34: 'New Jersey', 35: 'New Mexico', 36: 'New York', 37: 'North Carolina', 38: 'North Dakota', 39: 'Ohio', 40: 'Oklahoma', 41: 'Oregon', 42: 'Pennsylvania', 44: 'Rhode Island', 45: 'South Carolina', 46: 'South Dakota', 47: 'Tennessee', 48: 'Texas', 49: 'Utah', 50: 'Vermont', 51: 'Virginia', 53: 'Washington', 54: 'West Virginia', 55: 'Wisconsin', 56: 'Wyoming'}


,year,area,pct_delayed_healthcare,state_fips
0,2020,Alabama,39.200000,1
1,2020,Alaska,45.708333,2
2,2020,Arizona,41.016667,4
3,2020,Arkansas,40.575000,5
4,2020,California,41.116667,6
5,2020,Colorado,41.158333,8
6,2020,Connecticut,41.391667,9
7,2020,Delaware,38.766667,10
8,2020,District of Columbia,43.500000,11
9,2020,Florida,40.216667,12


In [309]:
hps.to_csv('../Data/Intermediate/hps_processed.csv')

In [3]:
path3 = '../Data/Clean/nonACSDemographics'
saipes = []
for file in os.listdir(path3):
    if not file.startswith('.'):
        saipes.append(pd.read_csv(os.path.join(path3, file), sep = ',', index_col = 0))
        print(file)

SAIPE2007_clean.csv
SAIPE2006_clean.csv
SAIPE2005_clean.csv
SAIPE2008_clean.csv
SAIPE2003_clean.csv
usdaERSCountyEmployment_clean.csv
SAIPE2004_clean.csv
SAIPE2009_clean.csv
SAIPE2002_clean.csv
census_total_pop_2000_2010_clean.csv


In [4]:
total = []
yrs = range(2002, 2009)
total.append(saipes[5])
total.append(saipes[9])
del(saipes[5])
del(saipes[-1])

for index, i in enumerate(saipes):
    #i['poverty_rate'] = pd.to_numeric(i['poverty_rate'], errors = 'coerce')
    i.replace('.', np.nan, inplace=True)
    i['poverty_rate'] = i['poverty_rate'].astype('float64').fillna(np.nan)
    i.iloc[:,1] = i.iloc[:,1].astype('float64').fillna(np.nan)
    #print(index, ':    ', i.columns)
    print(index, ':    ', i.dtypes)

0 :     poverty_rate              float64
per_capita_income_2007    float64
year                        int64
fips                      float64
dtype: object
1 :     poverty_rate              float64
per_capita_income_2006    float64
fips                      float64
year                        int64
dtype: object
2 :     poverty_rate              float64
per_capita_income_2005    float64
fips                      float64
year                        int64
dtype: object
3 :     poverty_rate              float64
per_capita_income_2008    float64
fips                      float64
year                        int64
dtype: object
4 :     poverty_rate              float64
per_capita_income_2003    float64
fips                        int64
year                        int64
dtype: object
5 :     poverty_rate              float64
per_capita_income_2004    float64
fips                        int64
year                        int64
dtype: object
6 :     poverty_rate              float64
per_capita

In [312]:
df3 = reduce(lambda  left,right: pd.merge(left, right, on=['year','fips', 'poverty_rate'],
                                            how='outer'), saipes)
total.append(df3)
df = reduce(lambda  left,right: pd.merge(left, right, on=['year','fips'],
                                            how='outer'), total)
print(df.columns)
df[df['fips']==45001].head()
print(df['year'].value_counts())

Index(['fips', 'year', 'employed_persons', 'unemployment_rate', 'total_pop',
       'poverty_rate', 'per_capita_income_2007', 'per_capita_income_2006',
       'per_capita_income_2005', 'per_capita_income_2008',
       'per_capita_income_2003', 'per_capita_income_2004',
       'per_capita_income_2009', 'per_capita_income_2002'],
      dtype='object')
2000    3228
2001    3228
2002    3228
2003    3228
2004    3228
2005    3228
2006    3228
2007    3228
2008    3228
2009    3228
Name: year, dtype: int64


In [313]:
df.to_csv('../Data/Intermediate/saipe0010.csv')

In [314]:
cdc_age = pd.read_csv('../Data/Clean/CDC/CDCSTDagesex20192020_clean.csv', index_col = 0)
cdc_age.columns =cdc_age.columns.str.replace('unknown', 'unknown_age')
list(cdc_age.columns)
cdc_age.to_csv('../Data/Clean/CDC/CDCSTDagesex20192020_clean.csv')

In [315]:
cdc_race = pd.read_csv('../Data/Intermediate/CDCSTDoverall2000_2020_processed.csv', index_col = 0)
cdc_race.columns = cdc_race.columns.str.replace('unknown', 'unknown_race')
list(cdc_race.columns)
#cdc_race.to_csv('../Data/Intermediate/CDCSTDoverall2000_2020_processed.csv')

['year',
 'fips',
 'cases_chlamydia',
 'cases_early_non-primary_non-secondary_syphilis',
 'cases_gonorrhea',
 'cases_primary_and_secondary_syphilis',
 'rate_per_100000_chlamydia',
 'rate_per_100000_early_non-primary_non-secondary_syphilis',
 'rate_per_100000_gonorrhea',
 'rate_per_100000_primary_and_secondary_syphilis',
 'cases_female_chlamydia_american_indian_alaska_native',
 'cases_female_chlamydia_asian',
 'cases_female_chlamydia_black_african_american',
 'cases_female_chlamydia_hispanic_latino',
 'cases_female_chlamydia_multiracial',
 'cases_female_chlamydia_native_hawaiian_other_pacific_islander',
 'cases_female_chlamydia_unknown_race',
 'cases_female_chlamydia_white',
 'cases_female_early_non-primary,_non-secondary_syphilis_american_indian_alaska_native',
 'cases_female_early_non-primary,_non-secondary_syphilis_asian',
 'cases_female_early_non-primary,_non-secondary_syphilis_black_african_american',
 'cases_female_early_non-primary,_non-secondary_syphilis_hispanic_latino',
 'case

In [316]:
total = []
folder = '../Data/Clean'

for dirs in os.listdir(folder):
    if not dirs.endswith(('.txt', 'checkpoints', 'HPS', 'ACS', 'USAFacts', 'graphics', 'ility')):
        #print(dirs)
        for files in os.listdir(os.path.join(folder,dirs)):
            if not files.endswith(('.txt', 'checkpoints', 'STD2019_2020_clean.csv')):
                total.append(pd.read_csv(os.path.join(folder,(os.path.join(dirs,files))), sep = ',', index_col = 0))
                print(os.path.join(folder, os.path.join(dirs, files)))
    
folder2 = '../Data/Intermediate'
for files2 in os.listdir(folder2):
    if not (files2.endswith(('.txt', 'checkpoints', 'hps_processed.csv'))):
            total.append(pd.read_csv(os.path.join(folder2,files2), sep = ',', index_col = 0))
            print(files2)
            print(pd.DataFrame(pd.read_csv(os.path.join(folder2,files2), sep = ',', index_col = 0)).shape)

../Data/Clean/CDC/disagRaceSexHIV2017_2020_clean.csv
../Data/Clean/CDC/CDCSTDagesex20192020_clean.csv
../Data/Clean/CDC/HIVcountyage_clean.csv
../Data/Clean/CDC/HIV_Diagnoses_Prevalence_2008-2021_clean.csv
CDCSTDoverall2000_2020_processed.csv
(67586, 130)
COVIDcounty_cases_processed.csv
(9426, 4)
ACSDP03_merge.csv
(35425, 31)
ACSDP05_merge.csv
(35425, 42)
saipe0010.csv
(32280, 14)


In [317]:
for index, i in enumerate(total):
    if 'year' in i.columns:
        if 'geography' in i.columns:
            i.drop(columns = 'geography', inplace = True)
            print('Done')
        if 'county_name' in i.columns:
            i.drop(columns = 'county_name' , inplace = True)
            print('done')

Done
Done
Done
Done
done


In [318]:
for index, i in enumerate(total):
    print(index)
    i = i[i['year'] >= 2002]
    print(i['year'].unique())
    for col in i.columns:
        print(col, ": ", i[col].dtypes)

0
[2017 2018 2019 2020]
year :  int64
fips :  int64
cases_female_hiv_diagnoses :  float64
cases_female_hiv_prevalence :  float64
cases_female_hiv_viral_suppression :  float64
cases_female_linkage_to_hiv_care :  float64
cases_female_prep_coverage_and_number_of_persons_prescribed :  float64
cases_female_receipt_of_hiv_medical_care :  float64
cases_male_hiv_diagnoses :  float64
cases_male_hiv_prevalence :  float64
cases_male_hiv_viral_suppression :  float64
cases_male_linkage_to_hiv_care :  float64
cases_male_prep_coverage_and_number_of_persons_prescribed :  float64
cases_male_receipt_of_hiv_medical_care :  float64
percent_female_hiv_viral_suppression :  float64
percent_female_linkage_to_hiv_care :  float64
percent_female_prep_coverage_and_number_of_persons_prescribed :  float64
percent_female_receipt_of_hiv_medical_care :  float64
percent_male_hiv_viral_suppression :  float64
percent_male_linkage_to_hiv_care :  float64
percent_male_prep_coverage_and_number_of_persons_prescribed :  float6

In [319]:
acs = pd.read_csv('../Data/Intermediate/ACSDP05_merge.csv', index_col = 0)
print(acs[(acs['year']==2018) & (acs['fips'] == 30037)]['fips'].value_counts())

30037    1
Name: fips, dtype: int64


In [321]:
cdcstd = pd.read_csv('../Data/Intermediate/CDCSTDoverall2000_2020_processed.csv',
                     index_col = 0)
print(cdcstd['year'].value_counts())

2020    3222
2019    3222
2011    3218
2018    3218
2017    3218
2016    3218
2015    3218
2014    3218
2013    3218
2012    3218
2000    3218
2001    3218
2009    3218
2008    3218
2007    3218
2006    3218
2005    3218
2004    3218
2003    3218
2002    3218
2010    3218
Name: year, dtype: int64
